In [1]:
#%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from loaddata import load_mnist
import model
import pickle

/home/artemy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = load_mnist()
tf.reset_default_graph()
n_epochs     = 50
n_sgd        = 128
n_eta_sgd    = 2000
#beta         = 0.05
beta_epoch   = 10
report_every = 10

# train model
sess=tf.Session()

n = model.Net(encoder_arch=[(512,'relu'),(512,'relu'),(2,'relu')], decoder_arch=[(512,'relu'),],
              trainable_sigma=False, log_sigma2=-4, init_beta=0.0)

In [5]:
for beta in np.linspace(0, 1, 21, endpoint=True):
    #with tf.Session() as sess:
    print("Doing beta=%0.5f" % beta)
    for mode in ('nlIB', 'VIB'):
        print("Doing %s" % mode)
        trainstep = n.nlIB_trainstep if mode == 'nlIB' else n.VIB_trainstep
        loss      = n.nlIB_loss      if mode == 'nlIB' else n.VIB_loss
        
        sess.run(tf.global_variables_initializer())

        n_mini_batches = int(len(data['train_labels']) / n_sgd)

        saved_data = []
        fname = 'sqIBv2-%s-%0.5f' % (mode, beta)

        for epoch in range(n_epochs):
            # randomize order of training data
            permutation = np.random.permutation(len(data['train_labels']))
            train_data = data['train_data'][permutation]
            train_labels = data['train_labels'][permutation]

            #if epoch == 0:
            if n.trainable_sigma:
                n.sigma_optimizer.minimize(sess, feed_dict={n.x: x_batch, n.y: y_batch})

            x_batch = train_data[:n_eta_sgd]
            #y_batch = train_labels[:n_eta_sgd]
            n.eta_optimizer.minimize(sess, feed_dict={n.x: x_batch})
            if epoch == beta_epoch:
                assign_op = n.beta.assign(beta)
                sess.run(assign_op)

            for batch in range(n_mini_batches):
                # sample mini-batch
                x_batch = train_data[batch * n_sgd:(1 + batch) * n_sgd]
                y_batch = train_labels[batch * n_sgd:(1 + batch) * n_sgd]

                cparams = {n.x: x_batch, n.y: y_batch}
                sess.run(trainstep, feed_dict=cparams)

            cur_beta, cur_log_sigma2, cur_log_eta2 = sess.run([n.beta, n.log_sigma2, n.log_eta2])

            tst_acc, tst_loss, tst_Ixt, tst_Iyt = \
                sess.run([n.accuracy, loss, n.Ixt, n.Iyt], feed_dict={n.x: data['test_data'][::10], n.y: data['test_labels'][::10]})
            trn_acc, trn_loss, trn_Ixt, trn_Iyt = \
                sess.run([n.accuracy, loss, n.Ixt, n.Iyt], feed_dict={n.x: data['train_data'][::10], n.y: data['train_labels'][::10]})

            cdata = {'epoch': epoch, 'beta': cur_beta, 'log_sigma2':cur_log_sigma2, 'log_eta2':cur_log_eta2,
                     'trn_acc': trn_acc, 'tst_acc': tst_acc, 'trn_loss': trn_loss, 'tst_loss': tst_loss,
                     'trn_Ixt': trn_Ixt, 'tst_Ixt': tst_Ixt, 'trn_Iyt' : trn_Iyt , 'tst_Iyt' : tst_Iyt}
            saved_data.append(cdata)

            with open('outdata/'+fname, 'wb') as fp:
                pickle.dump(saved_data, fp)

            if epoch % report_every == 0:
                print()
                print('epoch  : %d/%d | beta: %0.4f | noisevar: %0.4f | kernelwidth: %0.4f'%
                      (epoch+1, n_epochs, cur_beta, np.exp(cur_log_sigma2), np.exp(cur_log_eta2)))
                print('acc    : % 0.4f / % 0.4f' % (trn_acc, tst_acc))
                print('loss   : % 0.4f / % 0.4f' % (trn_loss, tst_loss))
                print('I(X;T) : % 0.4f / % 0.4f' % (trn_Ixt, tst_Ixt))
                print('I(Y;T) : % 0.4f / % 0.4f' % (trn_Iyt, tst_Iyt))

        print()
        print()

Doing beta=0.00000
Doing nlIB


AttributeError: 'Net' object has no attribute 'loss'

In [ ]:
# with tf.Session() as sess:
import seaborn as sns
plt.figure(figsize=(10,10))
x, y = data['test_data'], data['test_labels']
mx = sess.run(n.encoder[-1], feed_dict={n.x: x})
plt.scatter(mx[:,0],mx[:,1], s=4, c=np.argmax(y,axis=1), alpha=0.1)
plt.axis('off');

In [ ]:
mx